In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random as rn
from datetime import datetime

from tensorflow.keras import backend as K

In [19]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

seed=1234

# set the seed for random number generator
rn.seed(seed)

# Set the numpy seed
np.random.seed(seed)

# Set the random seed in tensorflow at graph level
#tf.set_random_seed(seed)
tf.random.set_seed(seed)

# We will be using CuDNN implementation of RNNs which already is non-reproducible
# So I am trying to get results as close as possible on different runs
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)




In [2]:
batch_size = 16

# Desired image dimensions
img_width = 200
img_height = 50
max_length = 6

In [3]:
characters ={'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [4]:
# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), mask_token=None
)


# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

2021-11-26 01:06:36.088967: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-26 01:06:36.089075: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-26 01:06:36.089162: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (siva-signalx): /proc/driver/nvidia/version does not exist
2021-11-26 01:06:36.090059: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-26 01:06:36.293019: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 240000000

In [5]:
def encode_single_sample(img_path, label):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # 6. Map the characters in label to numbers
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    return {"image": img, "label": label}

In [6]:
# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

In [7]:
os.getcwd()

'/home/siva/Downloads/captcha-recognition-pytorch-master'

# Loading saved model 

In [24]:
reconstructed_pred_model2 = keras.models.load_model("new_predict_model2")

# Predicting Captcha284

In [26]:
#all_image_paths[0:1]
test_img_path =['/home/siva/Downloads/captcha-recognition-pytorch-master/captcha_test/captcha284.png']
test_img_path[0:1]

#print(tf.rank(x_valid))
#print(tf.rank(test_img_path[0:1]))

validation_dataset1 = tf.data.Dataset.from_tensor_slices((test_img_path[0:1], ['']))
validation_dataset1 = (
    validation_dataset1.map(
        encode_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

for batch in validation_dataset1.take(1):
    #print(batch['image'])
    
    preds = reconstructed_pred_model2.predict(batch['image'])
    pred_texts = decode_batch_predictions(preds)

print(pred_texts)

['708046']
